<a href="https://colab.research.google.com/github/Tariqameenkhan/LangChain-RAG/blob/main/RAG_PDF_Multiples_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai  langchain-pinecone

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [3]:
!pip uninstall -y google-ai-generativelanguage


Found existing installation: google-ai-generativelanguage 0.6.16
Uninstalling google-ai-generativelanguage-0.6.16:
  Successfully uninstalled google-ai-generativelanguage-0.6.16


In [1]:
!pip install google-ai-generativelanguage==0.6.15


In [3]:
!pip install --upgrade langchain-text-splitters langchain-community langchain-google-genai langchain-pinecone


In [1]:
!pip install --upgrade google-generativeai


In [2]:
!pip install -qU pypdf

In [3]:
import os
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')
userdata.get('PINECONE_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Ensure your VertexAI credentials are configured

from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")



In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings.embed_query("Hello world")



In [17]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec
api_key = os.environ.get('PINECONE_API_KEY')

pc = Pinecone(api_key= api_key, environment='us-east-1')  # Use your Pinecone environment

index_name = "helloworld2"


pc.create_index(
      name=index_name,
      dimension=768, # Replace with your model dimensions
      metric="cosine", # Replace with your model metric
      spec=ServerlessSpec(
          cloud="aws",
          region="us-east-1"
      )
  )

index = pc.Index(index_name)

vector_store = PineconeVectorStore(embedding=embeddings, index=index)

In [18]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load multiple PDF documents
pdf_files = ["name.pdf", "details.pdf"]  # Replace with your actual filenames

documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents.extend(loader.load())  # Append documents from each PDF

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(f"Total document chunks: {len(docs)}")


Total document chunks: 2


In [19]:
from tqdm import tqdm

# Create embeddings and upload to Pinecone
vectors = []
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)
    doc_id = str(hash(doc.page_content))  # Generate a unique ID
    metadata = {"text": doc.page_content}
    vectors.append((doc_id, vector, metadata))

# Upsert all vectors to Pinecone in one batch
index.upsert(vectors=vectors)


100%|██████████| 2/2 [00:00<00:00,  7.27it/s]


{'upserted_count': 2}

In [20]:
from langchain.vectorstores import Pinecone

retriever = Pinecone(index=index, embedding=embeddings, text_key="text")

# Convert into retriever
# retriever = vector_store.as_retriever()

# Increase retrieval size
retriever = vector_store.as_retriever(search_kwargs={"k": 10})

In [22]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Other options: "map_reduce", "refine"
    retriever=retriever
)


In [23]:
# Run interactive query loop
while True:
    user_input = input("Ask something: ")
    if user_input.lower() == "quit":
        break
    else:
        response = qa_chain.run(user_input)
        print(response)

Ask something: show all data
Here's all the data provided:

* **Names:** Tariq, Ameen, Khan, Zain
* **Age:** 12
* **Hobby:** Football
* **Preference:** Loves cakes
Ask something: what is y hobby?
Your provided text states that your hobby is football.
Ask something: what is my name ?
I don't know.  The provided text gives several names (Tariq, Ameen, Khan, Zain), but doesn't state which one is yours.
Ask something: what is my age ?
Your age is 12.
Ask something: in food i love is ?
Cakes.
Ask something: quit
